In [22]:
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics

from sklearn.naive_bayes import MultinomialNB

import pandas as pd

In [23]:
GTC_V2 = pd.read_csv('https://raw.githubusercontent.com/MiriamSchirmer/genocide-transcript-corpus/main/Dataset_GTC-V2.csv', sep =';')
GTC_V2['tribunal'].value_counts()

tribunal
ICTY    19217
ICTR    17752
ECCC    15876
Name: count, dtype: int64

In [24]:
GTC_CAMB = GTC_V2.loc[GTC_V2['tribunal']=='ECCC']
GTC_CAMB.head(3)

,tribunal,id_transcript,case,accused,date,text,trauma,role,witnesses,n_witnesses,start,id_annotation,id_document,url
16862,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,P R O C E E D I N G S (Judges enter the courtr...,0,Court Proceedings,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,0,63b1f39c97ad59b4cfc5793d,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...
16863,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,MR. PRESIDENT: This morning the Trial Chamber ...,0,JudgeProc,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,51,63b1f3a397ad59b4cfc5793e,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...
16864,ECCC,E1/12.1,Case001,Kaing Guev Eav,2009-04-20 00:00:00 UTC,"THE GREFFIER: Your Honours, Mr. President, the...",0,Court Proceedings,"[Chan Voeun (KW-31), Chan Khan (KW-32)]",2,267,63b1f3a797ad59b4cfc5793f,63a60e2397ad59b4cfc571ec,http://www.eccc.gov.kh/sites/default/files/doc...


In [25]:
X = GTC_CAMB['text']
y = GTC_CAMB['trauma']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [26]:
X_train

21976    Q. Thank you. Now I would like to move back a ...
31254    Q. So you said that there were no arrests whil...
25828    Q. Then that mystery is solved. Thank you very...
28814    A. I was a child and I was taken to Phnom Penh...
28085                                        Q. Thank you.
                               ...                        
29985    And the third point regarding the torture and ...
20126              Q. What did he do between 1975 and '79?
26707    Her son, who was living in Phnom Penh, did not...
27661    A. I can draw a clear conclusion that it must ...
19594    A. No, I did not hear anything about torture. ...
Name: text, Length: 12700, dtype: object

In [41]:
text_pipeline = Pipeline([
     ('tokenize', CountVectorizer(max_features=100000, stop_words="english")),
     ('tfidf', TfidfTransformer(use_idf=True)),
     ('classifier', SGDClassifier())
    ])

In [42]:
text_pipeline.fit(X_train, y_train)

Pipeline(steps=[('tokenize',
                 CountVectorizer(max_features=100000, stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('classifier', SGDClassifier())])

In [43]:
y_pred_test = text_pipeline.predict(X_test)
y_pred_train = text_pipeline.predict(X_train)

In [48]:
y_train.value_counts(normalize=True)
#92% null accuracy if we just guess no trauma every time

trauma
0    0.923701
1    0.076299
Name: proportion, dtype: float64

In [44]:
metrics.accuracy_score(y_train, y_pred_train)

0.9595275590551181

In [45]:
metrics.accuracy_score(y_test, y_pred_test)

0.9423803526448362

In [46]:
print(metrics.classification_report(y_test, y_pred_test, zero_division=0))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      2943
           1       0.84      0.27      0.40       233

    accuracy                           0.94      3176
   macro avg       0.89      0.63      0.69      3176
weighted avg       0.94      0.94      0.93      3176

